### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import math
from math import sqrt
plt.style.use('seaborn-poster')

C:\Users\maria\AppData\Local\Temp\ipykernel_50980\700012485.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


### Custo máximo da energia elétrica

In [113]:
def custo_eletrico_banho(Sr, potencia_eletrica, custo_eletrico_kwh, tempo):
    """Calcula o custo da parte elétrica do banho.

    O custo da parte elétrica do banho é dado pela potência do chuveiro em KW multiplicado pela fração de utilização
    da resistência elétrica Sr, o custo do kWh em reais, e o tempo do banho em horas. Como o tempo é em minutos, 
    divide-se por 60. Como Sr é uma ação, seu valor é constante para toda a iteração.

    Argumentos:
        Sr (float): Seletor da resistência elétrica do tanque de aquecimento.
        potencial_eletrica (float): Potência elétrica do tanque de aquecimento (chuveiro) em kW.
        custo_eletrico_kwh (float): Custo do kWh da energia em reais por hora.
        tempo (float): Tempo da ação em minutos.

    Retorna:
        custo_eletrico_total (float): Custo da energia elétrica do banho em reais.
    """
    custo_eletrico_total = potencia_eletrica * Sr * custo_eletrico_kwh * tempo / 60

    return custo_eletrico_total

In [114]:
potencia_eletrica = 5.5
Sr = 1
custo_eletrico_kwh = 2
tempo = 14

custo_eletrico_max = custo_eletrico_banho(Sr, potencia_eletrica, custo_eletrico_kwh, tempo)
custo_eletrico_max

2.566666666666667

### Custo máximo de gás

In [115]:
def custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt):
    """Calcula o custo do gás do banho.

    A potência de um aquecedor a gás é dada em kcal/h. Considerando um rendimento de 86%, a potência últi será
    a potência multiplicada pelo rendimento. Para saber quantas kcal são fornecidas durante o banho, multiplica-se
    a potência útil pela quantidade de energia gasta de Sa (área da curva) e divide por 60 para tempo em horas.
    O pode calorífico do gás GLP é de 11750 kcal/kg. Se multiplicarmos esse valor pela quantidade de kcal gasta no banho,
    é possível obter a quantidade de gás em kg gasta no banho. Finalmente, o custo em reais é dado pela quantidade de gás em kg
    multiplicada pelo custo do gás em kg/reais.
    
    Referências: 
        https://conteudos.rinnai.com.br/vazao-aquecedor-de-agua-a-gas/#:~:text=A%20pot%C3%AAncia%20do%20aquecedor%20%C3%A9,hora%20(kcal%2Fh)
        https://www.supergasbras.com.br/super-blog/negocio/2021/qual-a-vantagem-do-poder-calorifico-do-glp#:~:text=GLP%3A%2011.750%20Kcal%2Fkg,G%C3%A1s%20Natural%3A%209.400%20Kcal%2Fm%C2%B3
    
    Argumentos:
        Sa (float): Seletor do aquecimento do boiler.
        potencial_aquecedor (float): Potência do aquecedor (boiler) em kcal/h.
        custo_gas_kg (float): Custo do kg do gás em reais por kg.
        tempo (float): Tempo da ação em minutos.
        df (float): Passo de tempo da simulação.

    Retorna:
        custo_gas_total (float): Custo do gás do banho em reais.
    """
    # Tempo em minutos que o aquecedor fica ligado (Sa = 1):
    # numero_vezes_aquecedor_ligado = np.count_nonzero(Sa == 1) # integrar também?
    # tempo_aquecedor_ligado = tempo * numero_vezes_aquecedor_ligado / (tempo / dt)

    # Quantidade de Sa utilizado:
    Sa_utilizado = np.trapz(y=Sa, dx=dt)

    # Potência útil do aquecedor em kcal/h:
    rendimento = 1
    potencia_util = potencia_aquecedor * rendimento

    # Quantidade de kcal fornecida durante o banho:
    # kcal_fornecida_no_banho = potencia_util * tempo_aquecedor_ligado / 60
    kcal_fornecida_no_banho = potencia_util * Sa_utilizado / 60

    # Poder calorífico do gás em kcal/kg (GLP):
    kg_equivalente_kcal = 11750 

    # Quantidade de gás gasta durante o banho:
    quantidade_gas_kg = kcal_fornecida_no_banho / kg_equivalente_kcal 

    # Custo do gás:
    custo_gas_total = custo_gas_kg * quantidade_gas_kg

    return custo_gas_total 

In [117]:
potencia_aquecedor = 29000
custo_gas_kg = 3
dt = 0.01
Sa = np.repeat(1, 1400)

custo_gas_max = custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt)
custo_gas_max

1.7264255319148933

In [118]:
a = np.arange(start=0, stop=14 + 0.07, step=0.01, dtype="float")
a

array([0.000e+00, 1.000e-02, 2.000e-02, ..., 1.404e+01, 1.405e+01,
       1.406e+01])

In [119]:
a.size

1407

### Custo máximo água

In [120]:
def modelo_valvula_saida(xs):
    """Modelo para a vazão válvula de saída (Fs).

    Argumentos:
        xs (float): Abertura da válvula de saída.

    Retorna:
        Fs (float): Vazão da válvula de saída.
    """
    n = 5 * (xs ** 3) * (sqrt(30)) * sqrt(-15 * (xs ** 6) + sqrt(6625 * (xs ** 12) + 640 * (xs ** 6) + 16))
    d = 20 * (xs ** 6) + 1

    Fs = n / d
    return Fs

In [121]:
def custo_agua_banho(Fs, custo_agua_m3, tempo):
    """Calcula o custo de água do banho.

    A quantidade de litros gasta em um banho é dada pela vazão em L/min multiplicada pelo tempo em minutos do banho.
    Depois, divide-se a quantidade de litros por 1000, para obter a quantidade em m3 gasta durante o banho.
    Multiplicando esse valor pelo custo da água em m3/reais, obtém-se o custo em reais da água gasta durante o banho.

    Argumentos:
        xs (float): Vazão da válvula de saída.
        custo_agua_m3 (float): Custo da água em m3 por reais. 
        tempo (float): Tempo da ação em minutos.

    Retorna:
        custo_agua_total: Custo da água do banho em reais.
    """
    # integrar a vazão da água
    custo_agua_total = ((Fs * tempo ) / 1000) * custo_agua_m3 

    return custo_agua_total

In [122]:
xs = 0.99
Fs = modelo_valvula_saida(0.99)
Fs

10.92317131464205

In [123]:
custo_agua_m3 = 4
tempo = 14

custo_agua_max = custo_agua_banho(Fs, custo_agua_m3, tempo)
custo_agua_max

0.6116975936199548

In [124]:
xs = 0.1
Fs = modelo_valvula_saida(0.1)
custo_agua_m3 = 4
tempo = 14

custo_agua_min = custo_agua_banho(Fs, custo_agua_m3, tempo)
custo_agua_min

0.0030672098991168556

### Recompensa
    reward = c1 * self.iqb + c2 * 1 / (self.custo_eletrico / self.custo_eletrico_maximo_14_min)

In [125]:
def calculo_iqb(Ts, Fs):
    """Calcula o índice de qualidade do banho (iqb).
    
    Argumentos:
        Ts (float): Temperatura de saída do chuveiro.
        Fs (float): Vazão da válvula de saída.

    Retorna:
        iqb (float): Índice de qualidade do banho.
    """
    iqb = (1 / math.e) * math.exp((1 - ((Ts - 38 + 0.02 * Fs) / 2) ** 2) * np.power((0.506 + math.log10(math.log10((10000 * np.sqrt(Fs)) / (10 + Fs + 0.004 * np.power(Fs, 4))))), 20))

    if np.isnan(iqb) or iqb == None or np.isinf(abs(iqb)) or iqb < 0:
        iqb = 0
    if iqb > 1:
        iqb = 1
        
    return iqb

In [126]:
def calculo_recompensa(Ts, Fs, Sr, Sa):
    
    iqb = calculo_iqb(Ts, Fs)
    print(f"IQB: {iqb}")
    
    potencia_eletrica = 5.5
    custo_eletrico_kwh = 2
    tempo = 2
    custo_eletrico = custo_eletrico_banho(Sr, potencia_eletrica, custo_eletrico_kwh, tempo)
    print(f"Custo elétrico: {custo_eletrico}")
    
    potencia_aquecedor = 29000
    custo_gas_kg = 3
    dt = 0.01
    custo_gas = custo_gas_banho(Sa, potencia_aquecedor, custo_gas_kg, dt)
    print(f"Custo gás: {custo_gas}")
    
    custo_agua_m3 = 4
    tempo = 2
    Fs = 4.5
    tempo = 2
    custo_agua = custo_agua_banho(Fs, custo_agua_m3, tempo)
    print(f"Custo água: {custo_agua}")
    
    if custo_eletrico == 0 and custo_gas != 0:
        reward = 3 * iqb + 4 + 0.01 * (1 / (custo_gas / custo_gas_max))
        
    if custo_eletrico != 0 and custo_gas == 0:
        reward = 3 * iqb + 0.05 * (1 / (custo_eletrico / custo_eletrico_max))
        
    if custo_eletrico == 0 and custo_gas == 0:
        reward = 3 * iqb
        
    if custo_eletrico != 0 and custo_gas != 0:
        reward = 3 * iqb + 0.05 * (1 / (custo_eletrico / custo_eletrico_max)) + 0.01 * (1 / (custo_gas / custo_gas_max))
    print(f"Recompensa: {reward}")
    print("")

In [127]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 4.5
    Sr = i / 10
    Sa1 = np.repeat(1, 100 + 10 * i)
    Sa2 = np.repeat(0, 100 - 10 * i)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9583508400688454
Custo elétrico: 0.0
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 7.015655535281913

IQB: 0.9583508400688454
Custo elétrico: 0.03666666666666667
Custo gás: 0.13512765957446804
Custo água: 0.036
Recompensa: 6.502815077284161

IQB: 0.9583508400688454
Custo elétrico: 0.07333333333333333
Custo gás: 0.14746808510638296
Custo água: 0.036
Recompensa: 4.7421236499136485

IQB: 0.9583508400688454
Custo elétrico: 0.11
Custo gás: 0.15980851063829787
Custo água: 0.036
Recompensa: 4.149750074904091

IQB: 0.9583508400688454
Custo elétrico: 0.14666666666666667
Custo gás: 0.17214893617021276
Custo água: 0.036
Recompensa: 3.8503392585577907

IQB: 0.9583508400688454
Custo elétrico: 0.18333333333333332
Custo gás: 0.18448936170212765
Custo água: 0.036
Recompensa: 3.668631115524262

IQB: 0.9583508400688454
Custo elétrico: 0.22
Custo gás: 0.19682978723404254
Custo água: 0.036
Recompensa: 3.5460974522859514

IQB: 0.9583508400688454
Custo elétrico: 0.25666666666666665
Cus

In [129]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 4.5
    Sr = i / 10
    Sa1 = np.repeat(1, 1)
    Sa2 = np.repeat(0, 199)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9583508400688454
Custo elétrico: 0.0
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 34.855052520206534

IQB: 0.9583508400688454
Custo elétrico: 0.03666666666666667
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 34.355052520206534

IQB: 0.9583508400688454
Custo elétrico: 0.07333333333333333
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 32.605052520206534

IQB: 0.9583508400688454
Custo elétrico: 0.11
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 32.0217191868732

IQB: 0.9583508400688454
Custo elétrico: 0.14666666666666667
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 31.730052520206534

IQB: 0.9583508400688454
Custo elétrico: 0.18333333333333332
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 31.555052520206534

IQB: 0.9583508400688454
Custo elétrico: 0.22
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 31.438385853539867

IQB: 0.9583508400688454
Custo elétrico: 0.25666

In [130]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 4.5
    Sr = i / 10
    Sa1 = np.repeat(0, 100)
    Sa2 = np.repeat(0, 100)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9583508400688454
Custo elétrico: 0.0
Custo gás: 0.0
Custo água: 0.036
Recompensa: 2.8750525202065362

IQB: 0.9583508400688454
Custo elétrico: 0.03666666666666667
Custo gás: 0.0
Custo água: 0.036
Recompensa: 6.375052520206536

IQB: 0.9583508400688454
Custo elétrico: 0.07333333333333333
Custo gás: 0.0
Custo água: 0.036
Recompensa: 4.625052520206536

IQB: 0.9583508400688454
Custo elétrico: 0.11
Custo gás: 0.0
Custo água: 0.036
Recompensa: 4.041719186873203

IQB: 0.9583508400688454
Custo elétrico: 0.14666666666666667
Custo gás: 0.0
Custo água: 0.036
Recompensa: 3.7500525202065362

IQB: 0.9583508400688454
Custo elétrico: 0.18333333333333332
Custo gás: 0.0
Custo água: 0.036
Recompensa: 3.5750525202065364

IQB: 0.9583508400688454
Custo elétrico: 0.22
Custo gás: 0.0
Custo água: 0.036
Recompensa: 3.4583858535398697

IQB: 0.9583508400688454
Custo elétrico: 0.25666666666666665
Custo gás: 0.0
Custo água: 0.036
Recompensa: 3.3750525202065362

IQB: 0.9583508400688454
Custo elétrico: 0.2933333

In [132]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 4.5
    Sr = i / 10
    Sa1 = np.repeat(1, 100)
    Sa2 = np.repeat(0, 100)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9583508400688454
Custo elétrico: 0.0
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 7.015655535281913

IQB: 0.9583508400688454
Custo elétrico: 0.03666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 6.515655535281913

IQB: 0.9583508400688454
Custo elétrico: 0.07333333333333333
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.765655535281913

IQB: 0.9583508400688454
Custo elétrico: 0.11
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.18232220194858

IQB: 0.9583508400688454
Custo elétrico: 0.14666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.890655535281913

IQB: 0.9583508400688454
Custo elétrico: 0.18333333333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.7156555352819134

IQB: 0.9583508400688454
Custo elétrico: 0.22
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.5989888686152467

IQB: 0.9583508400688454
Custo elétrico: 0.25666666666666665
Custo

In [133]:
for i in range(0, 11):
    Ts = 37.5
    Fs = 5.5
    Sr = i / 10
    Sa1 = np.repeat(1, 100)
    Sa2 = np.repeat(0, 100)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.8883307912041434
Custo elétrico: 0.0
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 6.805595388687807

IQB: 0.8883307912041434
Custo elétrico: 0.03666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 6.305595388687807

IQB: 0.8883307912041434
Custo elétrico: 0.07333333333333333
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.555595388687807

IQB: 0.8883307912041434
Custo elétrico: 0.11
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.9722620553544736

IQB: 0.8883307912041434
Custo elétrico: 0.14666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.680595388687807

IQB: 0.8883307912041434
Custo elétrico: 0.18333333333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.5055953886878073

IQB: 0.8883307912041434
Custo elétrico: 0.22
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.3889287220211406

IQB: 0.8883307912041434
Custo elétrico: 0.25666666666666665
Cus

In [134]:
for i in range(0, 11):
    Ts = 35.0
    Fs = 3.5
    Sr = i / 10
    Sa1 = np.repeat(1, 10)
    Sa2 = np.repeat(0, 190)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.11546500686379108
Custo elétrico: 0.0
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 5.819026599538741

IQB: 0.11546500686379108
Custo elétrico: 0.03666666666666667
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 5.319026599538741

IQB: 0.11546500686379108
Custo elétrico: 0.07333333333333333
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 3.569026599538742

IQB: 0.11546500686379108
Custo elétrico: 0.11
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 2.9856932662054083

IQB: 0.11546500686379108
Custo elétrico: 0.14666666666666667
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 2.694026599538742

IQB: 0.11546500686379108
Custo elétrico: 0.18333333333333332
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 2.5190265995387415

IQB: 0.11546500686379108
Custo elétrico: 0.22
Custo gás: 0.011723404255319148
Custo água: 0.036
Recompensa: 2.4023599328720753

IQB: 0.11546500686379108
Custo elétrico: 0.256666

In [135]:
for i in range(0, 11):
    Ts = 35.0
    Fs = 3.5
    Sr = i / 10
    Sa1 = np.repeat(1, 1)
    Sa2 = np.repeat(0, 199)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.11546500686379108
Custo elétrico: 0.0
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 32.32639502059137

IQB: 0.11546500686379108
Custo elétrico: 0.03666666666666667
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 31.82639502059137

IQB: 0.11546500686379108
Custo elétrico: 0.07333333333333333
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 30.07639502059137

IQB: 0.11546500686379108
Custo elétrico: 0.11
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 29.493061687258038

IQB: 0.11546500686379108
Custo elétrico: 0.14666666666666667
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 29.20139502059137

IQB: 0.11546500686379108
Custo elétrico: 0.18333333333333332
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 29.02639502059137

IQB: 0.11546500686379108
Custo elétrico: 0.22
Custo gás: 0.0006170212765957447
Custo água: 0.036
Recompensa: 28.909728353924702

IQB: 0.11546500686379108
Custo elétrico: 0.

In [136]:
for i in range(0, 11):
    Ts = 38.0
    Fs = 4.5
    Sr = i / 10
    Sa1 = np.repeat(1, 100)
    Sa2 = np.repeat(0, 100)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.9974403748301958
Custo elétrico: 0.0
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 7.132924139565964

IQB: 0.9974403748301958
Custo elétrico: 0.03666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 6.632924139565964

IQB: 0.9974403748301958
Custo elétrico: 0.07333333333333333
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.882924139565964

IQB: 0.9974403748301958
Custo elétrico: 0.11
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.299590806232631

IQB: 0.9974403748301958
Custo elétrico: 0.14666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 4.007924139565964

IQB: 0.9974403748301958
Custo elétrico: 0.18333333333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.8329241395659643

IQB: 0.9974403748301958
Custo elétrico: 0.22
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.7162574728992976

IQB: 0.9974403748301958
Custo elétrico: 0.25666666666666665
Cust

In [137]:
for i in range(0, 11):
    Ts = 36.5
    Fs = 4.0
    Sr = i / 10
    Sa1 = np.repeat(1, 100)
    Sa2 = np.repeat(0, 100)
    Sa = np.concatenate((Sa1, Sa2), axis=None)
    
    calculo_recompensa(Ts, Fs, Sr, Sa)

IQB: 0.6085607529878673
Custo elétrico: 0.0
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 5.966285274038978

IQB: 0.6085607529878673
Custo elétrico: 0.03666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 5.466285274038978

IQB: 0.6085607529878673
Custo elétrico: 0.07333333333333333
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.7162852740389787

IQB: 0.6085607529878673
Custo elétrico: 0.11
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 3.132951940705645

IQB: 0.6085607529878673
Custo elétrico: 0.14666666666666667
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 2.8412852740389787

IQB: 0.6085607529878673
Custo elétrico: 0.18333333333333332
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 2.666285274038979

IQB: 0.6085607529878673
Custo elétrico: 0.22
Custo gás: 0.12278723404255319
Custo água: 0.036
Recompensa: 2.549618607372312

IQB: 0.6085607529878673
Custo elétrico: 0.25666666666666665
Cust